In [5]:
## INSTALL PACKAGE
!pip install -q markdown openai gdown xformers ctransformers tokenizers transformers accelerate langchain chainlit sentence_transformers chromadb unstructured PyPDF2 pypdf bitsandbytes faiss_cpu faiss_gpu huggingface_hub hf_transfer optimum -q
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  -q # Use cu117 if on CUDA 11.7

In [7]:
# LOAD LIBRARY
from torch import cuda, bfloat16, float16
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GenerationConfig, pipeline, TextStreamer

In [ ]:
%%bash
# Download model
mkdir -p Llama-7B-Chat-GPTQ
HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download TheBloke/Llama-2-7B-Chat-GPTQ --local-dir Llama-7B-Chat-GPTQ --local-dir-use-symlinks False

In [8]:
# 1. 情緒分析
###  model https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english
from transformers import pipeline
generator = pipeline("sentiment-analysis")
generator("I've been waiting for a HuggingFace course my whole life.")
generator(["I've been waiting for a HuggingFace course my whole life.", "I hate this so much!"])

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9598046541213989},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [9]:
# 2. 零訓練樣本分類
# model https://huggingface.co/facebook/bart-large-mnli
from transformers import pipeline
generator = pipeline("zero-shot-classification")
generator("This is a course about the Transformers library",candidate_labels=["education", "politics", "business"])

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'sequence': 'This is a course about the Transformers library',
 'labels': ['education', 'business', 'politics'],
 'scores': [0.8445951342582703, 0.1119769886136055, 0.04342784732580185]}

In [10]:
# 3. 文字生成 text-generation
# model https://huggingface.co/gpt2
from transformers import pipeline
generator = pipeline(task="text-generation")
generator("In this course, we will teach you how to")

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to develop a basic understanding of the language and basic concepts of linguistics and linguistics. We will show you how to create and create an HTML5 text file that can be copied into your browser. We'}]

In [12]:
# 4. 自動問答
# 自動問答pipeline 可以根據給定的上下文回答問題，例如：
# model https://huggingface.co/distilbert-base-cased-distilled-squad)
from transformers import pipeline
generator = pipeline("question-answering")
generator(question="Where am I form?",context="My name is Sylvain and I in Taiwan")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

{'score': 0.9241172075271606, 'start': 28, 'end': 34, 'answer': 'Taiwan'}

In [13]:
#5. 自動摘要
# 自動摘要pipeline 旨在將長文本壓縮成短文本，並且還要盡可能保留原文的主要訊息，例如：
# model https://huggingface.co/sshleifer/distilbart-cnn-12-6
from transformers import pipeline
generator = pipeline("summarization")
generator(
    """
    America has changed dramatically during recent years. Not only has the number of 
    graduates in traditional engineering disciplines such as mechanical, civil, 
    electrical, chemical, and aeronautical engineering declined, but in most of 
    the premier American universities engineering curricula now concentrate on 
    and encourage largely the study of engineering science. As a result, there 
    are declining offerings in engineering subjects dealing with infrastructure, 
    the environment, and related issues, and greater concentration on high 
    technology subjects, largely supporting increasingly complex scientific 
    developments. While the latter is important, it should not be at the expense 
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other 
    industrial countries in Europe and Asia, continue to encourage and advance 
    the teaching of engineering. Both China and India, respectively, graduate 
    six and eight times as many traditional engineers as does the United States. 
    Other industrial countries at minimum maintain their output, while America 
    suffers an increasingly serious decline in the number of engineering graduates 
    and a lack of well-educated engineers.
    """
)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

[{'summary_text': ' America has changed dramatically during recent years . The number of engineering graduates in the U.S. has declined in traditional engineering disciplines such as mechanical, civil, electrical, chemical, and aeronautical engineering . Rapidly developing economies such as China and India, as well as other industrial countries in Europe and Asia, continue to encourage and advance engineering .'}]

In [14]:
# 6. 圖像分類
# model google/vit-base-patch16-224
from transformers import pipeline
vision_classifier = pipeline(task="image-classification")
vision_classifier(
    images="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg"
)

No model was supplied, defaulted to google/vit-base-patch16-224 and revision 5dca96d (https://huggingface.co/google/vit-base-patch16-224).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

[{'score': 0.4403029680252075, 'label': 'lynx, catamount'},
 {'score': 0.03433407098054886,
  'label': 'cougar, puma, catamount, mountain lion, painter, panther, Felis concolor'},
 {'score': 0.03214794397354126,
  'label': 'snow leopard, ounce, Panthera uncia'},
 {'score': 0.023539068177342415, 'label': 'Egyptian cat'},
 {'score': 0.023034071549773216, 'label': 'tiger cat'}]